# Stein's Paradox Notebook

### ref: https://www.statslab.cam.ac.uk/~rjs57/SteinParadox.pdf
### questions / suggestions --> Adam Kraft adk@mit.edu
### this notebook grew out of conversations I had about Stein's Paradox with Mike Fleder and Josh Joseph

## Why

Because Stein's paradox is utterly mind blowing. I don't rememer where I was when Kennedy was shot, likely because I didn't exist then. But I remember where I was when Mike told me about Stein's paradox... if you haven't heard about it, whoa, make sure you're sitting down ;-)

But why put so much effort into simulating this? There are lots of interesting stats and probability puzzles that blew my mind initially. The monty hall problem is an example. For reference, in the monty hall problem there are 3 doors. Behind one is a new car. behind the other two are goats. A game show contestant picks one door at random. Monty, the host, then reveals a goat behind another door (he knows where the car is, and so he knows not to reveal it to you). The contestant must now decide: switch doors or choose to open the one they chose at first. The answer is that it's always better to switch. When I first heard about it as a kid, I thought it was a lie: both choices seemed equally good. I didn't believe the math teacher---I thought, screw that, I'll write a simulation to prove that I'm right. So I did, and in the process found out *why* my intuition failed me. For the monty hall problem, you can get great intuition by messing with the numbers. Suppose there were 100 doors instead of 3, and 99 of them have goats. You pick a door, then monty reveals 98 goats behind all but one of the remaining doors. Now, it's obvious that your chances are better (or worse if you like goats) if you switch. Wheras your odds of finding a car with the door you initially picked cannot increase from 1/100, the odds if you switch are now obviously much higher (99/100) because Monty has just given up a lot of information.

I tried hard to develop the same flavor of intuition for Stein's paradox and have so far failed. This notebook is for messing with the numbers until I, with some luck, develop a better intuition for Stein's insane puzzle.

## What

Stein's paradox takes a little bit more setup than the Monty Hall problem. Suppose we have 3 or more independent random variables. It adds emphasis to the mind-blowing aspect of this problem to think of these variables as obviously unrelated things: baseball stats, birthrates in China, Martian weather. All we know about them is that they're each drawn from some normal distribution: $X_i \sim \mathcal{N}(\theta_i,1)$. For now, don't worry about the fact that the variance is 1 or even that everything's a normal distribution. With some work, a lot of distributions can be transformed to fit the preconditions of this paradox. What we want is an estimator, $\hat{\theta} = f(X)$, to approximate the true means $\theta = <\theta_1, \theta_2, \theta_3, ... \theta_p>$.

We of course want a good esimator, so we'll define a loss function $L(\hat{\theta},\theta)$ as a way of evaluating guesses at our distribution of interest's vector of means. The loss function could be squared euclidian distance, or absolute error (taxicab distance) or some other metric. In principle, this loss could be any way of describing estimation error: it represents the penalty for error in guessing this vector of means, and so there is value to us in minimizing it. A very popular choice of loss function is mean squared error: proportional to squared euclidean distance between our estimated means and the true means. Since the estimator $\hat{\theta}$ depends on random observations, our loss value will be random too. To take this randomness out of the problem and be able to compare estimators to one another deterministically, we define risk, $R(\hat{\theta},\theta) = E[L(\hat{\theta},\theta)]$ as the expeted loss given some function for obtaining $\hat{\theta}$ from random obervations of $X$.

In order to reason about relative risk of different estimators, statisticians define a concept of **dominance**. They say an estimator $\hat{\theta}$ **strictly dominates** $\tilde{\theta}$ if $R(\hat{\theta},\theta) \le R(\tilde{\theta},\theta)$ for all $\theta$, and the inequality is strict for some $\theta$. That is to say, for any value $\theta$ could take, the risk, i.e. expected loss, of using estimator $\hat{\theta}$ is less than or equal to the risk of using estimator $\tilde{\theta}$. Furthermore, there is at least some $\theta$ for which the risk of $\hat{\theta}$ is strictly less than $\tilde{\theta}$. Clearly, if we want to minimize risk, se should never use an estimator $\tilde{\theta}$ if we can show that it is strictly dominated by another estimator $\hat{\theta}$. Any estimator that we can show to be strictly dominated is called **inadmissible**. If we can show that an estimator $\hat{\theta}$ is not strictly dominated by any other estimator, we call $\hat{\theta}$ **admissible**. So far, so good.

Now back to our variables: baseball, birthrates, Mars weather. Suppose we have one sample from each and we want to try to guess the means. If we consider any one variable on its own, the intuitive answer of using the sample itself as an estimate of that one variable's mean is admissible. Using the samples to estimate the means has advantages: it's ubniased (because $E[X_i] = \theta_i$) and it turns out to be the maximum likelihood esitmator. But, **if we're concerned with minimizing the overall risk of all three variables combined, this intuitive unbiased estimator is inadmissible!**. That's right: in order to minimize risk with the MSE loss function, the following function, called the James Stein estimator, strictly dominates the simple unbiased estimator discussed above.
$$\hat{\theta}^{JS}(X) = \left( 1 - \frac{p-2}{\lVert X \rVert^2} \right)X$$
What makes this seem so crazy is that to jointly estimate all of these obviously-independent variables, we have to scale each one by a factor that depends on all of the variables! Obviously, there is some slight of hand here. Chinese birth rates and baseball stats give us no info about weather on mars, and, sure enough, if we want to estimate the mean of some weather phenomenon on mars by itself we had better not pay attention to the other variables. But if our goal is to have the smalles error, measured as mean-squared error, of all three means, we have to contract each variable toward the origin (in fact, toward any point in the space) by a factor that depends on all three variables!

As a first step, let's generate some data and confirm that this is true.



In [2]:
import numpy as np

def generate_true_means(p=3,lower=-10,upper=10):
    """
    generate p means, each sampled from a uniform distribution, 
    that will be used as our latent distribution
    """
    return np.random.uniform(lower,upper,(p,))

def sample_gaussian(theta,n_samples):
    """
    given true means, sample n_samples from multivariate gaussian in which 
    components are independent and each component has unit variance
    """
    p = theta.shape[-1]
    cov = np.eye(p)
    samples = np.random.multivariate_normal(theta,cov,n_samples)
    return samples

def squared_euclidean_loss(theta_hat,theta):
    #note, this can operate on a matrix where each row is a sample
    #this idiom is used throughout the code.
    return np.sum((theta_hat-theta)**2,axis=-1,keepdims=True) 

def estimator_unbiased(X):
    theta_hat = X
    return theta_hat

def estimator_JS(X):
    p = X.shape[-1]
    contraction_coefficients = (1 - (p-2)/np.sum(X**2,axis=-1,keepdims=True))
    theta_hat = X * contraction_coefficients
    return theta_hat


def estimate_risk(theta, sample_fn, estimator_fn, loss_fn, n_trials=1000000):
    """
    empirically estimate risk as defined in the intro text above, by averaging loss
    over n_trials trials. in each trial, use sample_fn to draw a sample from distribution
    represented by theta. use estimator on the resulting sample to esimate theta_hat.
    use loss_fn to compute L(theta_hat,theta). average that loss over the trials.
    """
    samples = sample_fn(theta,n_trials)
    theta_hats = estimator_fn(samples)
    thetas = np.array([theta]*n_trials)
    losses = loss_fn(theta_hats,thetas)
    return np.mean(losses)


In [3]:
def run_sim(loss_fcn=squared_euclidean_loss,
            n_trials=1000000,
            latent_var_generator=generate_true_means,
            distribution_sampler=sample_gaussian,
            n_sims=10):
    js_dominates=True
    for i in range(n_sims):
        theta = latent_var_generator()
        risk_unbiased = estimate_risk(theta,distribution_sampler,estimator_unbiased,loss_fcn,n_trials=n_trials)
        risk_js = estimate_risk(theta,distribution_sampler,estimator_JS,loss_fcn,n_trials=n_trials)
        print("unbiased risk:",risk_unbiased,"js risk:",risk_js,"js is lower risk?",(risk_js<=risk_unbiased))
        js_dominates = js_dominates and (risk_js<=risk_unbiased)
    print("js dominates?",js_dominates)


In [4]:
run_sim()

unbiased risk: 3.00088927749 js risk: 2.98719447199 js is lower risk? True
unbiased risk: 3.00086239039 js risk: 2.98723636473 js is lower risk? True
unbiased risk: 2.99378001684 js risk: 2.99085101306 js is lower risk? True
unbiased risk: 3.00094276595 js risk: 2.99273013766 js is lower risk? True
unbiased risk: 2.99789847212 js risk: 2.99346748123 js is lower risk? True
unbiased risk: 3.00892378088 js risk: 2.99227537952 js is lower risk? True
unbiased risk: 3.00379337308 js risk: 2.96208382447 js is lower risk? True
unbiased risk: 2.99982450288 js risk: 2.99571244285 js is lower risk? True
unbiased risk: 3.00284039924 js risk: 2.98707630012 js is lower risk? True
unbiased risk: 3.00084652433 js risk: 2.98607221596 js is lower risk? True
js dominates? True


### Need n_trials to be large to observe the effect
the effect of JS is subtle: with 1 million samples used to compute risk, we still don't always find that $\hat{\theta}^{JS}(X)$ dominates. But it definitely does; increasing the number of samples shows that.

Ok, now how about if we use absolute error instead of euclidean error?

In [5]:
def absolute_loss(theta_hat,theta):
    return np.sum(np.abs(theta_hat-theta),axis=-1,keepdims=True)

In [6]:
run_sim(loss_fcn=absolute_loss)

unbiased risk: 2.39183596013 js risk: 2.39156104315 js is lower risk? True
unbiased risk: 2.3932891185 js risk: 2.38987689935 js is lower risk? True
unbiased risk: 2.39266103859 js risk: 2.38090047583 js is lower risk? True
unbiased risk: 2.39366459899 js risk: 2.38992987083 js is lower risk? True
unbiased risk: 2.39493366916 js risk: 2.38988140346 js is lower risk? True
unbiased risk: 2.39464171226 js risk: 2.38720531667 js is lower risk? True
unbiased risk: 2.39379725966 js risk: 2.39091555726 js is lower risk? True
unbiased risk: 2.39386097798 js risk: 2.3900399469 js is lower risk? True
unbiased risk: 2.39326008862 js risk: 2.38916527002 js is lower risk? True
unbiased risk: 2.39391636417 js risk: 2.38288141662 js is lower risk? True
js dominates? True


### Absolute loss (L1 norm) doesn't break it!
even if we use absolute loss, JS still appears to dominate the unbiased estimator.

can we choose a loss fcn that break it? I bet max error will...

In [7]:
def max_loss(theta_hat,theta):
    return np.max(np.abs(theta_hat-theta),axis=-1,keepdims=True)

In [8]:
run_sim(loss_fcn=max_loss)

unbiased risk: 1.32589846645 js risk: 1.32257469273 js is lower risk? True
unbiased risk: 1.32560571552 js risk: 1.32483549578 js is lower risk? True
unbiased risk: 1.32668257798 js risk: 1.32318757835 js is lower risk? True
unbiased risk: 1.32674595245 js risk: 1.32572464257 js is lower risk? True
unbiased risk: 1.32740197526 js risk: 1.32472382784 js is lower risk? True
unbiased risk: 1.32691695518 js risk: 1.32534133488 js is lower risk? True
unbiased risk: 1.32588688321 js risk: 1.32513683092 js is lower risk? True
unbiased risk: 1.32664037246 js risk: 1.32420508808 js is lower risk? True
unbiased risk: 1.32597955085 js risk: 1.3226652968 js is lower risk? True
unbiased risk: 1.32661782029 js risk: 1.27987217512 js is lower risk? True
js dominates? True


### Well, shit!

even using max loss, JS appears to dominate!  WTF, I thought it was more sensitive to loss fcn!
as a sanity check, a loss that only depends on one variable should no longer dominate 

In [9]:
def zero_component_loss(theta_hat,theta):
    return np.abs(theta[...,0:1]-theta_hat[...,0:1])

In [10]:
run_sim(loss_fcn=zero_component_loss)

unbiased risk: 0.797794413041 js risk: 0.796090273946 js is lower risk? True
unbiased risk: 0.797754454687 js risk: 0.793869528743 js is lower risk? True
unbiased risk: 0.797130980098 js risk: 0.802510505514 js is lower risk? False
unbiased risk: 0.799344325352 js risk: 0.815856334357 js is lower risk? False
unbiased risk: 0.797072925785 js risk: 0.792398960101 js is lower risk? True
unbiased risk: 0.798510339494 js risk: 0.767414397852 js is lower risk? True
unbiased risk: 0.79634023774 js risk: 0.756385006513 js is lower risk? True
unbiased risk: 0.797436871659 js risk: 0.798273515625 js is lower risk? False
unbiased risk: 0.798532452665 js risk: 0.814827689939 js is lower risk? False
unbiased risk: 0.797724871549 js risk: 0.796965147395 js is lower risk? True
js dominates? False


### sanity check passed

as expected, if we only pay attention to 1 variable instead of all three, JS no longer dominates the unbiased estimator.

### one thing still bugs me, though:
the word "contraction" gets thrown around a lot when people talk about Stein's. Intuitively, it makes sense that the JS estimator is pulling points toward the origin; essentially "shrinking" the distribution by pulling it towards that point. **But, what if the point is already close to the origin?** Then, it seems to me like the JS estimator flings it farther away from the origin.

In code:

In [11]:
def always_at_origin(p=3,**unused):
    """
    a "generator" that always returns the origin
    """
    return np.array([0.0]*p)

run_sim(latent_var_generator=always_at_origin)

unbiased risk: 2.99952735967 js risk: 1.99140469751 js is lower risk? True
unbiased risk: 2.99704824875 js risk: 1.97946907713 js is lower risk? True
unbiased risk: 3.00122664109 js risk: 1.98033629352 js is lower risk? True
unbiased risk: 3.00299505872 js risk: 1.99215083773 js is lower risk? True
unbiased risk: 2.99977517598 js risk: 1.99354835489 js is lower risk? True
unbiased risk: 2.99897774223 js risk: 1.98324805558 js is lower risk? True
unbiased risk: 3.00219979325 js risk: 2.00807801336 js is lower risk? True
unbiased risk: 3.00070596007 js risk: 1.97878894996 js is lower risk? True
unbiased risk: 3.00662480304 js risk: 2.00450831202 js is lower risk? True
unbiased risk: 3.00092273183 js risk: 1.98410254482 js is lower risk? True
js dominates? True


### Huh.

So, I guess if a distribution has variance 1 there are still sufficiently many points farther than 1 unit away from the origin that risk is reduced, even though we're taking the points that are near the origin and flinging them far away. 

Let me just confirm that:

In [12]:
def sample_sharper_peak(theta,n_samples):
    """
    use a tighter variance than our standard variance of 1.
    
    Note: this violates a precondition of Stein's; I'm just trying
    to verify that this in fact breaks Stein's by flinging too many
    points far away from the origin.
    """
    p = theta.shape[-1]
    cov = np.eye(p) * 0.1 #variance of 0.1 instead of 1
    samples = np.random.multivariate_normal(theta,cov,n_samples)
    return samples

run_sim(latent_var_generator=always_at_origin, distribution_sampler=sample_sharper_peak)

unbiased risk: 0.299905824055 js risk: 8.46171014387 js is lower risk? False
unbiased risk: 0.29960184041 js risk: 8.05846945773 js is lower risk? False
unbiased risk: 0.300311550277 js risk: 8.0648158097 js is lower risk? False
unbiased risk: 0.299863434905 js risk: 8.16929531697 js is lower risk? False
unbiased risk: 0.300149820818 js risk: 8.22669856405 js is lower risk? False
unbiased risk: 0.299994543816 js risk: 8.0897086506 js is lower risk? False
unbiased risk: 0.29966219171 js risk: 9.29839160854 js is lower risk? False
unbiased risk: 0.300162710453 js risk: 8.10705148974 js is lower risk? False
unbiased risk: 0.300270731672 js risk: 8.13825081243 js is lower risk? False
unbiased risk: 0.300086443939 js risk: 8.10956589179 js is lower risk? False
js dominates? False


### Good.

That is totally what I'd expect. when the distribution has a sharp peak and most samples are less than distance 1 from the origin, the $1-\frac{p-2}{\lVert X \rVert^2}$ multiplier in the JS estimator is negative with magnitude greater than 1 for most points X, and so takes X and flings it far away from the origin.

So this is a pretty good confirmation of what I said before, that, when the variance is 1, even if the distribution mean is at the origin, there are still sufficiently many points farther than 1 unit away from the origin that risk is reduced, even though we are taking the points that are very near the origin and flinging them far away. That is, the effect of flinging these points is sufficiently weak not to matter, when variance is 1 (as it's required to be).

### Another suspicion I have about Stein's paradox is that the effect should get more obvious as we increase dimensionality. 

Let's try to confirm that suspicion:

In [13]:
def generate_10D_means(**kwargs):
    """
    generate 10 means, each sampled from a uniform distribution, 
    that will be used as our latent distribution
    """
    return generate_true_means(p=10,**kwargs)


run_sim(latent_var_generator=generate_10D_means)

unbiased risk: 9.99663410834 js risk: 9.76935178408 js is lower risk? True
unbiased risk: 9.98682816783 js risk: 9.87257510371 js is lower risk? True
unbiased risk: 9.9915495043 js risk: 9.81215470962 js is lower risk? True
unbiased risk: 9.99636994345 js risk: 9.70283798501 js is lower risk? True
unbiased risk: 10.0019451909 js risk: 9.83629326997 js is lower risk? True
unbiased risk: 9.9994251385 js risk: 9.82419352993 js is lower risk? True
unbiased risk: 9.99307221852 js risk: 9.83020585513 js is lower risk? True
unbiased risk: 9.9962595945 js risk: 9.76423630337 js is lower risk? True
unbiased risk: 10.0112459953 js risk: 9.86938830891 js is lower risk? True
unbiased risk: 10.0007884936 js risk: 9.889532119 js is lower risk? True
js dominates? True


### Yes, 

the numbers seems a lot more favorable in 10D. That is, there is like a 2% difference between JS risk and unbiased risk in 10D, whereas there was only about a 0.5% avg difference in risk for 3 dimensions. How about 100D?

In [14]:
def generate_100D_means(**kwargs):
    """
    generate 100 means, each sampled from a uniform distribution, 
    that will be used as our latent distribution
    """
    return generate_true_means(p=100,**kwargs)


run_sim(latent_var_generator=generate_100D_means,n_sims=1)#only run once; it'll take a lot longer

unbiased risk: 100.020037107 js risk: 97.2700626695 js is lower risk? True
js dominates? True


### That's super weird. 

The difference in risk between JS and unbiases seems to converge on 3%-ish. I would have thought it'd get exponentially better as we increase dimension!!!

what about some other loss function (not MSE)? let's try mean absolute error and max error:

In [18]:
print("absolute loss:")
run_sim(latent_var_generator=generate_100D_means,n_sims=1,loss_fcn=absolute_loss)
print("max loss:")
run_sim(latent_var_generator=generate_100D_means,n_sims=1,loss_fcn=max_loss)

absolute loss:
unbiased risk: 79.7793436666 js risk: 78.7143297313 js is lower risk? True
js dominates? True
max loss:
unbiased risk: 2.74675141527 js risk: 2.71021062272 js is lower risk? True
js dominates? True


### Wow...
2 takeaways: JS still improves over the unbiased estimator, and the improvement is still small, like 2-3%

# Once upon a time, at Mamalehs,
Mike, Josh and I tried to come up with a geometric interpretation of Stein's paradox. The thought was, somehow the volume of sample space in which estimates are improved by contracting toward the origin exceeds the volume of space in which estimates are made worse by contracting toward the origin. Or else maybe the volume in which things are improved is improved more than the volume in which things are made worse. Or some combination of those effects.

If the geometric interpretation can be made intuitive somehow (like with a picture) then even though the stats interpretation is still insanity-inducing, we can at least think of it in terms of the geometric metaphor. Maybe, we can put it all behind us like a bad dream: could it be that Stein's paradox is just an artifact of symmetry? That is, because our distributions are all rotationally symmetric about all axes that pass through their means (because independent & all unit variance) maybe it's just a trick about the geometry of spheres (hyperspheres/balls) that makes stein's paradox work at all. then we can rest easier; distributions with less symmetry are less likely to be affected by this paradox.

IIRC, we had drawn something like this on a napkin:
![alt text](stein_imgs/F1.png "Figure 1")

The JS esitimator moves $X$ toward the origin along the line through $X$ and the origin (except in the degenerate case discussed and simulated above, where it flings $X$ away, but we can probs ignore that case).

If $X$ was farther from the origin along that line than point $q$, as shown on the figure, then the movement brings $X$ closer to $\theta$ until $X$ passes $q$. Then, once it's past $q$, $X$'s movement toward the origin starts to increase the distance between $X$ and $q$. It's worth noting that the constraints on point $q$ are such that all possible $q$ define a sphere of diameter $\lVert\theta\rVert$, centered at point $\frac{\theta}{2}$:

![alt text](stein_imgs/f2.png "Figure 2")

Points inside that sphere are made farther from $\theta$ by their movement toward the origin. Points outside the sphere are brought closer to $\theta$ by movement toward the origin. If we look at how this sphere intersects the probability distribution with mean $\theta$, we see that it always encloses less than half of the distribution, due to its curvature:

![alt text](stein_imgs/f3.png "Figure 3")

That is, the fuzzy grey blob centered at $\theta$ represents the probability distribution. because the sphere intersects the center of the distribution and is curved, less than half of the total integrated density (mass) of the distribution will be inside the sphere. 

**In high dimensions, this effect will be profound: as dimensions increase, the fraction of the mass contained by the sphere decays exponentially toward zero, I think.**

**BUT! It's not true that the points inside the sphere are made worse estimates by JS, and the ones outside made better.** The boundary where JS transitions from making points better estimates to making points worse estimates lies outside the sphere. This is because JS moves points by a finite distance of $\left(\frac{p-2}{\lVert X \rVert^2} \right)X$ toward the origin. so the actual boundary between points improved by JS to points not improved by JS is some distance away from the surface of the sphere, along the vector pointing away from the origin. So, that means that the mean of the distribution, $\theta$ lies within the region of space that is made worse by applying the JS estimator. Maybe this fact can help to explain why there is no JS-like estimator for 2 dimensions. Like, maybe if you try to construct one you will end up with equal volumes in the made-worse and made-better parts of the world, because of this slight offset from the curved surface.

<p style="text-align: center;"> ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ </p>

OK, maybe think about this quantitatively now so we can simulate something.

The distance of point $q$ from the origin is:

$$\lVert\theta\rVert\cos(\alpha) = \frac{X \cdot \theta}{\lVert X \rVert}$$

The coordinates of point $q$ are

$$\left(\frac{X \cdot \theta}{\lVert X \rVert^2}\right)X$$

This gets us somewhere, maybe. all points $X$ that get closer to $q$, after the JS estimator has had its way with them, will result in improved estimates of $\theta$ under JS than if we used the points themselves as estimates of the mean. That is, the points that improve under Stein's are those points for which:

$$\left\lVert\hat{\theta}^{JS}(X) - \left(\frac{X \cdot \theta}{\lVert X \rVert^2}\right)X\right\rVert \lt \left\lVert X - \left(\frac{X \cdot \theta}{\lVert X \rVert^2}\right)X \right\rVert$$,

$$\left\lVert \left( 1 - \frac{p-2}{\lVert X \rVert^2} \right)X - \left(\frac{X \cdot \theta}{\lVert X \rVert^2}\right)X\right\rVert \lt \left\lVert X - \left(\frac{X \cdot \theta}{\lVert X \rVert^2}\right)X \right\rVert$$,

$$\left\lVert \left( 1 - \frac{p-2}{\lVert X \rVert^2} - \frac{X \cdot \theta}{\lVert X \rVert^2}\right)X\right\rVert \lt \left\lVert \left(1 - \frac{X \cdot \theta}{\lVert X \rVert^2}\right)X \right\rVert$$,

$$\left| 1 - \frac{p-2}{\lVert X \rVert^2} - \frac{X \cdot \theta}{\lVert X \rVert^2}\right|\lVert X\rVert \lt \left|1 - \frac{X \cdot \theta}{\lVert X \rVert^2}\right|\lVert X\rVert$$,

$$\left| 1 - \frac{p-2}{\lVert X \rVert^2} - \frac{X \cdot \theta}{\lVert X \rVert^2}\right| \lt \left|1 - \frac{X \cdot \theta}{\lVert X \rVert^2}\right|$$,

$$\left| 1 - \frac{p-2 - X \cdot \theta}{\lVert X \rVert^2}\right| \lt \left|1 - \frac{X \cdot \theta}{\lVert X \rVert^2}\right|$$,

next step: simulate with different values of p to see what proportion of points are improved.